In [17]:
text_file = "eng-ban.txt"
# To open the text file we need to encode the text. Here, we use 'utf8' encoding
with open(text_file, encoding="utf8") as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, bangla = line.split(" >>> ")
    bangla = "[start] " + bangla + " [end]"
    text_pairs.append((bangla, english))

In [18]:
import random
for i in range(5):
    print(random.choice(text_pairs));

('[start] আমি উনাকে চিনি না. [end]', 'I do not know him.')
('[start] একুশে ফেব্রুয়ারি আন্তর্জাতিক মাতৃভাষা দিবস. [end]', '21st of February is the International Mother Language Day.')
('[start] বাংলা আমার মাতৃভাষা. [end]', 'Bangla is my mother tounge.')
